<a href="https://colab.research.google.com/github/sain78/app.py/blob/main/doctor_REG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install gradio scikit-learn pandas

In [23]:
import gradio as gr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import make_pipeline

# Apne dataset ko load karen (CSV file ke path ko update karen)
df = pd.read_csv('/content/expanded_doctor_suggestion_dataset.csv')

# Preprocessing - Vectorizing 'Disease' aur 'Doctor Ka Shuba' ke liye
X = df['Disease']  # Disease column (Bimari ki jagah Disease use kiya gaya hai)
y = df['Doctor Ka Shuba']  # Doctor specialty ka column

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model pipeline (Naive Bayes with CountVectorizer)
model = make_pipeline(CountVectorizer(), MultinomialNB())
model.fit(X_train, y_train)

# Function to recommend doctor and hospital based on illness input
def recommend_doctor(disease):  # 'illness' ko 'disease' se replace kiya gaya hai
    # Exact matching - Bimari ko lowercase mein convert kar ke match karte hain
    filtered_data = df[df['Disease'].str.lower() == disease.lower()]

    # Check if any matching doctors are found
    if filtered_data.empty:
        return "No matching doctor found for this disease. Please check the spelling or try another disease."

    # Prepare the output with multiple doctors if available
    recommendations = []
    for index, row in filtered_data.iterrows():
        doctor_name = row['Doctor Ka Naam']
        doctor_specialty = row['Doctor Ka Shuba']
        hospital = row['Hospital']
        location = row['Location']
        experience = row['Tajurba (saal mein)']

        recommendations.append(f"Doctor: {doctor_name}\nSpecialty: {doctor_specialty}\nHospital: {hospital}\nLocation: {location}\nExperience: {experience} years\n")

    # Return all doctor recommendations
    return "\n".join(recommendations)

# Gradio Interface
interface = gr.Interface(fn=recommend_doctor,
                         inputs=gr.Textbox(label="Enter your Disease (Bimari)"),
                         outputs=gr.Textbox(label="Recommended Doctor and Hospital"))

# Launch the Gradio interface
interface.launch()
